In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

from light_correction import LightCorrection
from motion_deblur import MotionDeblur, MotionDeblurConfig
from quality_enhancement import QualityEnhancer, QualityEnhancementConfig

plt.rcParams["figure.figsize"] = (10, 5)

In [ ]:
# helper to display BGR images as RGB

def show_side_by_side(before, after, title_before="Before", title_after="After"):
    """
    Display two BGR images side-by-side as RGB using matplotlib.
    """
    before_rgb = cv2.cvtColor(before, cv2.COLOR_BGR2RGB)
    after_rgb = cv2.cvtColor(after, cv2.COLOR_BGR2RGB)

    fig, axes = plt.subplots(1, 2)
    axes[0].imshow(before_rgb)
    axes[0].set_title(title_before)
    axes[0].axis("off")

    axes[1].imshow(after_rgb)
    axes[1].set_title(title_after)
    axes[1].axis("off")

    plt.tight_layout()
    plt.show()

In [ ]:
# light distortion
light_img_path = "light_example_2.jpg"

light_img = cv2.imread(light_img_path)
if light_img is None:
    raise FileNotFoundError(f"Could not load image at {light_img_path}")

light_corrector = LightCorrection()
light_fixed = light_corrector(light_img)

show_side_by_side(
    light_img,
    light_fixed,
    title_before="Light Distorted (Input)",
    title_after="After LightCorrection",
)

In [ ]:
# motion blur
motion_img_path = "motion_example.jpg"
motion_img = cv2.imread(motion_img_path)
if motion_img is None:
    raise FileNotFoundError(f"Could not load image at {motion_img_path}")

config = MotionDeblurConfig(
    use_thread_pool=True,
    use_process_pool=False,
    max_workers=16,
    lengths=(9, 15, 21, 31),
    angles=tuple(float(i) for i in range(0, 180, 15)),
    iterations=100,        
    blur_threshold=9999.0,    
    apply_always=True,    
    final_unsharp_amount=0.6,
    final_unsharp_sigma=1.0,
)

deblurrer = MotionDeblur(config)
motion_deblurred = deblurrer(motion_img)

show_side_by_side(
    motion_img,
    motion_deblurred,
    title_before="Motion Blurred (Input)",
    title_after="After MotionDeblur",
)

In [ ]:
# low quality
quality_img_path = "quality_example.jpg"

quality_img = cv2.imread(quality_img_path)
if quality_img is None:
    raise FileNotFoundError(f"Could not load image at {quality_img_path}")

quality_config = QualityEnhancementConfig(
    target_min_side=640,
    unsharp_amount=0.8,
    unsharp_sigma=1.2,
)
enhancer = QualityEnhancer(quality_config)

quality_enhanced = enhancer(quality_img)

show_side_by_side(
    quality_img,
    quality_enhanced,
    title_before="Low-quality / Rescreenshot (Input)",
    title_after="After QualityEnhancer",
)